# 小学校出前授業アンケート分析

このノートブックでは、社会実装の出前授業で収集したアンケートデータの分析を行います。

In [ ]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud
import japanize_matplotlib
import nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

# グラフの日本語表示設定
plt.rcParams['font.sans-serif'] = ['IPAexGothic']
plt.rcParams['axes.unicode_minus'] = False

# Seabornのスタイル設定
sns.set_style('whitegrid')
sns.set_palette('Set2')

## 1. データの読み込み

In [ ]:
# CSVファイルの読み込み（ファイルパスを適宜変更してください）
# df = pd.read_csv('survey_data.csv', encoding='utf-8')

# サンプルデータの作成（実際のデータがない場合のテスト用）
sample_data = {
    '学年': ['3年', '3年', '4年', '4年', '5年', '5年', '6年', '6年'] * 10,
    '性別': ['男', '女', '男', '女', '男', '女', '男', '女'] * 10,
    '授業の面白さ': np.random.choice(['とても面白かった', '面白かった', '普通', 'あまり面白くなかった'], 80),
    '理解度': np.random.choice(['よく理解できた', '理解できた', '少し理解できた', '理解できなかった'], 80),
    '興味の変化': np.random.choice(['とても興味を持った', '興味を持った', '変わらない', '興味が減った'], 80),
    '感想': [
        'プログラミングが楽しかった',
        'ロボットが動くのがすごかった',
        '難しかったけど面白かった',
        'もっとやりたい',
        'コンピュータの仕組みがわかった'
    ] * 16
}

df = pd.DataFrame(sample_data)
print(f"データ件数: {len(df)}件")
df.head()

## 2. 基本統計情報

In [ ]:
# 基本情報の確認
print("=== データ型の確認 ===")
print(df.dtypes)
print("\n=== 欠損値の確認 ===")
print(df.isnull().sum())
print("\n=== 各カテゴリの分布 ===")
for col in df.select_dtypes(include='object').columns:
    print(f"\n{col}:")
    print(df[col].value_counts())

## 3. 可視化分析

### 3.1 学年・性別分布

In [ ]:
# 学年別・性別の分布
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# 学年分布
df['学年'].value_counts().sort_index().plot(kind='bar', ax=ax1)
ax1.set_title('学年別回答者数', fontsize=14)
ax1.set_xlabel('学年')
ax1.set_ylabel('人数')

# 性別分布
df['性別'].value_counts().plot(kind='pie', autopct='%1.1f%%', ax=ax2)
ax2.set_title('性別分布', fontsize=14)
ax2.set_ylabel('')

plt.tight_layout()
plt.show()

### 3.2 授業評価の分析

In [ ]:
# 授業評価の可視化
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 授業の面白さ
df['授業の面白さ'].value_counts().plot(kind='bar', ax=axes[0,0], color='skyblue')
axes[0,0].set_title('授業の面白さ評価', fontsize=14)
axes[0,0].set_xlabel('評価')
axes[0,0].set_ylabel('人数')

# 理解度
df['理解度'].value_counts().plot(kind='bar', ax=axes[0,1], color='lightgreen')
axes[0,1].set_title('理解度評価', fontsize=14)
axes[0,1].set_xlabel('評価')
axes[0,1].set_ylabel('人数')

# 興味の変化
df['興味の変化'].value_counts().plot(kind='bar', ax=axes[1,0], color='coral')
axes[1,0].set_title('興味の変化', fontsize=14)
axes[1,0].set_xlabel('評価')
axes[1,0].set_ylabel('人数')

# 学年別の授業評価
cross_tab = pd.crosstab(df['学年'], df['授業の面白さ'])
cross_tab.plot(kind='bar', ax=axes[1,1], stacked=True)
axes[1,1].set_title('学年別授業評価', fontsize=14)
axes[1,1].set_xlabel('学年')
axes[1,1].set_ylabel('人数')
axes[1,1].legend(title='授業の面白さ', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

### 3.3 インタラクティブな可視化（Plotly）

In [ ]:
# サンキーダイアグラムで関係性を可視化
# 学年 -> 授業の面白さ -> 興味の変化 の流れを表示

# データの準備
sankey_data = df.groupby(['学年', '授業の面白さ', '興味の変化']).size().reset_index(name='count')

# ノードの作成
all_nodes = list(df['学年'].unique()) + list(df['授業の面白さ'].unique()) + list(df['興味の変化'].unique())
node_dict = {node: i for i, node in enumerate(all_nodes)}

# リンクの作成
source = []
target = []
value = []

for _, row in sankey_data.iterrows():
    source.append(node_dict[row['学年']])
    target.append(node_dict[row['授業の面白さ']])
    value.append(row['count'])
    
    source.append(node_dict[row['授業の面白さ']])
    target.append(node_dict[row['興味の変化']])
    value.append(row['count'])

# サンキーダイアグラムの作成
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=all_nodes
    ),
    link=dict(
        source=source,
        target=target,
        value=value
    )
)])

fig.update_layout(title_text="学年・授業評価・興味の変化の関係", font_size=10)
fig.show()

## 4. テキスト分析（感想）

In [ ]:
# 感想のワードクラウド作成
text = ' '.join(df['感想'].dropna())

# 日本語フォントの設定
font_path = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'

# ワードクラウドの生成
try:
    wordcloud = WordCloud(
        width=800, 
        height=400,
        background_color='white',
        font_path=font_path,
        max_words=100,
        relative_scaling=0.5,
        min_font_size=10
    ).generate(text)
    
    plt.figure(figsize=(15, 8))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('感想のワードクラウド', fontsize=16)
    plt.show()
except:
    print("ワードクラウドの生成に失敗しました。フォントパスを確認してください。")
    
# 頻出単語の抽出
print("\n=== 感想に含まれる頻出フレーズ ===")
from collections import Counter
words = text.split()
word_counts = Counter(words)
for word, count in word_counts.most_common(10):
    print(f"{word}: {count}回")

## 5. 詳細分析

In [ ]:
# ヒートマップで相関を確認
# カテゴリカルデータを数値に変換
le = LabelEncoder()
df_encoded = df.copy()
for col in ['学年', '性別', '授業の面白さ', '理解度', '興味の変化']:
    df_encoded[col + '_encoded'] = le.fit_transform(df[col])

# 相関行列の計算
corr_cols = [col for col in df_encoded.columns if '_encoded' in col]
corr_matrix = df_encoded[corr_cols].corr()

# ヒートマップの描画
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0,
            xticklabels=[col.replace('_encoded', '') for col in corr_cols],
            yticklabels=[col.replace('_encoded', '') for col in corr_cols])
plt.title('各項目間の相関関係', fontsize=14)
plt.show()

## 6. レポート用集計

In [ ]:
# サマリーレポートの作成
print("=== アンケート分析サマリー ===")
print(f"\n総回答数: {len(df)}件")
print(f"\n【学年別内訳】")
for grade, count in df['学年'].value_counts().sort_index().items():
    print(f"{grade}: {count}名 ({count/len(df)*100:.1f}%)")

print(f"\n【授業評価】")
positive_rate = len(df[df['授業の面白さ'].isin(['とても面白かった', '面白かった'])]) / len(df) * 100
print(f"肯定的評価（面白かった以上）: {positive_rate:.1f}%")

understanding_rate = len(df[df['理解度'].isin(['よく理解できた', '理解できた'])]) / len(df) * 100
print(f"理解度（理解できた以上）: {understanding_rate:.1f}%")

interest_rate = len(df[df['興味の変化'].isin(['とても興味を持った', '興味を持った'])]) / len(df) * 100
print(f"興味の向上（興味を持った以上）: {interest_rate:.1f}%")

# 結果をCSVで保存
summary_df = pd.DataFrame({
    '項目': ['肯定的評価率', '理解度', '興味の向上率'],
    'パーセンテージ': [positive_rate, understanding_rate, interest_rate]
})
summary_df.to_csv('survey_summary.csv', index=False, encoding='utf-8-sig')
print("\nサマリーをsurvey_summary.csvに保存しました。")

## 7. データエクスポート機能

In [ ]:
# 分析結果のエクスポート
# 学年別集計
grade_summary = df.groupby('学年').agg({
    '性別': 'count',
    '授業の面白さ': lambda x: (x.isin(['とても面白かった', '面白かった'])).sum()
}).rename(columns={'性別': '回答者数', '授業の面白さ': '肯定的評価数'})
grade_summary['肯定的評価率'] = grade_summary['肯定的評価数'] / grade_summary['回答者数'] * 100

print("=== 学年別集計結果 ===")
print(grade_summary)

# Excelファイルとして保存（複数シート）
with pd.ExcelWriter('survey_analysis_results.xlsx', engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='生データ', index=False)
    grade_summary.to_excel(writer, sheet_name='学年別集計')
    summary_df.to_excel(writer, sheet_name='全体サマリー', index=False)

print("\n分析結果をsurvey_analysis_results.xlsxに保存しました。")